In [10]:
import numpy as np
import tensorflow as tf
import queue
import pickle
import zipfile
import collections
import math
import time

from collections import Counter

In [11]:
from train_util import *
from preprocessing import *
from layer import *
from model import *

In [12]:
#test_word2vec()

1. Preprocessing the data
===

In [13]:
#parameters
vocab_size = 100000
threshold = 1e-5
embedding_size = 200
context_size = 5
architecture = 'cbow'
use_loss = 'Negative_Sampling'
sample_size = 20
lr = 1e-3
epochs = 10

In [14]:
words = preprocess('text8.zip')

In [15]:
words, word_lst, word_dict, word_reverse_dict, words_index, unigram_dict = total_sampling(file_data=words, subsampling=True, threshold=1e-5)

making dictionary :  4.3428263664245605
unknown count :  4.3108580112457275
subsampling time :  12.028310775756836
unigram dictionary :  0.04686903953552246


2. negative Sampling and Hierarchical Softmax
====

In [16]:
sample_table = Unigram_Table(vocab_size,sample_size,unigram_dict)
huffman_encode = Huffman_encoding(unigram_dict)
huffman_encode.encoding()
node_lst, node_dict, reverse_node_dict = huffman_encode.make_node_dict()

Filling Unigram Table


In [17]:
architectures = ['cbow', 'skipgram']
use_loss = ['Negative_Sampling','Hierarchical_Softmax']
learning_rate = {'Softmax':1e-5, 'Negative_Sampling':0.15, 'Hierarchical_Softmax':0.15}

In [19]:
for e in range(epochs):
    for architecture in architectures:
        for loss_type in use_loss:
            print ('Testing Model :%s, loss : %s' % (architecture, loss_type))
            model = Word2Vec(vocab_size, embedding_size, context_size, architecture=architecture, verbose=len(words_index)//100,
                            use_loss=loss_type, learning_rate = learning_rate[loss_type], sample_table=sample_table, encoder=node_dict,
                            decay_step=len(words_index)//20, decay_rate=0.95, neg_samples=sample_size)
            model.train(words_index)
        
    # model save
    np.save('./results/%d_%s_%s_input.npy' % (e, architecture, loss_type), model.params['inputVectors']) # 0.15
    np.save('./results/%d_%s_%s_output.npy'% (e, architecture, loss_type), model.params['outputVectors']) # 0.15 

Testing Model :cbow, loss : Negative_Sampling
idx: 47963, loss: 14.532425, time: 9.036454, 
idx: 95926, loss: 14.531736, time: 8.244503, 


KeyboardInterrupt: 